In [2]:
import conllu

In [3]:
import pandas as pd

In [4]:
with open('text_to_test_control_cases.conllu', "r", encoding="utf-8") as f:
    data = f.read()
    pets_depression_sents = conllu.parse(data)

In [79]:
sum([len(sent) for sent in pets_depression_sents])

3911

In [110]:
def get_control_using_syntax(conllu_sentences):
    found_cases = pd.DataFrame({'sample': [], 'context': []})
    for sentence in conllu_sentences:
        nouns = sentence.filter(upostag='NOUN')
        sentence_forms = ' '.join([i['form'] for i in sentence])
        if nouns:
            #print(nouns)
            for noun in nouns:
                try:
                    case = noun['feats']['Case']
                    if case != 'Nom':
                        parent = sentence[noun['head']-1]
                        if parent['upostag'] == 'VERB':
                            preposition_candidates = sentence.filter(upostag='ADP', head=noun['id'])
                            if preposition_candidates:
                                sample = parent['form'] + ' ' + preposition_candidates[0]['form'] + ' ' + noun['form']
                                with_preposition = True
                            else:
                                sample = parent['form'] + ' ' + noun['form']
                                with_preposition = False
                            if case != 'Nom':
                                found_cases.loc[len(found_cases)] = [sample, sentence_forms]
                except Exception as e:
                    pass
    return found_cases 


In [9]:
def count_equal_results(conllu_sentences):
    n = 0
    for sentence in conllu_sentences:
        nouns = sentence.filter(upostag='NOUN')
        sentence_forms = ' '.join([i['form'] for i in sentence])
        if nouns:
            for noun in nouns:
                case = noun['feats']['Case']
                if case != 'Nom':
                    parent = sentence[noun['head']-1]
                    if parent['upostag'] == 'VERB':
                        if noun['id'] == parent['id'] + 1:
                            n += 1
    return n

In [10]:
count_equal_results(pets_depression_sents)

87

In [16]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 1000)

In [112]:
syntax_found = get_control_using_syntax(pets_depression_sents)
syntax_found 

,sample,context
0,анализируются В статье,"В статье анализируются свидетельства о депрессивных расстройствах животных , сделанные этологами и психиатрами в XX ..."
1,сделанные этологами,"В статье анализируются свидетельства о депрессивных расстройствах животных , сделанные этологами и психиатрами в XX ..."
2,сделанные в веке,"В статье анализируются свидетельства о депрессивных расстройствах животных , сделанные этологами и психиатрами в XX ..."
3,ставит во главу,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к..."
4,угла критерии,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к..."
5,становится подходом,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
6,становится к объяснению,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
7,рассматривалось в рамках,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
8,имеющее природу,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
9,могут в организме,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."


Всего строк: 399, из них 212 правильных (включая причастия)

Смешанный подход: всё то же самое, только нас интересуют только слова, которые стоят перед глаголом

In [127]:
syntax_found['context'][226]

'Поведение макак в яме отчаяния , по словам Харлова , менялось в сторону того , что «мы могли бы назвать депрессией» : животное переставало двигаться , сидело в скрючившейся позе , иногда обхватив себя лапами .'

In [5]:
def get_control_using_syntax_verb_postposition_only(conllu_sentences):
    found_cases = pd.DataFrame({'sample': [], 'context': []})
    for sentence in conllu_sentences:
        nouns = sentence.filter(upostag='NOUN')
        sentence_forms = ' '.join([i['form'] for i in sentence])
        if nouns:
            #print(nouns)
            for noun in nouns:
                try:
                    case = noun['feats']['Case']
                    if case != 'Nom':
                        parent = sentence[noun['head']-1]
                        if parent['upostag'] == 'VERB' and parent['id'] > noun['id']:
                            preposition_candidates = sentence.filter(upostag='ADP', head=noun['id'])
                            if preposition_candidates:
                                sample = parent['form'] + ' ' + preposition_candidates[0]['form'] + ' ' + noun['form']
                                with_preposition = True
                            else:
                                sample = parent['form'] + ' ' + noun['form']
                                with_preposition = False
                            if case != 'Nom':
                                found_cases.loc[len(found_cases)] = [sample, sentence_forms]
                except Exception as e:
                    pass
    return found_cases 

In [6]:
syntax_found_verb_postposition = get_control_using_syntax_verb_postposition_only(pets_depression_sents)

In [14]:
syntax_found_verb_postposition

,sample,context
0,анализируются В статье,"В статье анализируются свидетельства о депрессивных расстройствах животных , сделанные этологами и психиатрами в XX ..."
1,становится подходом,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
2,становится к объяснению,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
3,рассматривалось в рамках,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
4,могут в организме,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."
5,могут животных,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."
6,работал В контексте,В контексте психических расстройств перевод «животное — человек — животное» работал неудовлетворительно : не хватало...
7,соотносились «симптомы»,"Наблюдаемые «симптомы» плохо соотносились с критериями расстройства , предполагающими обращение к субъектным пережив..."
8,развивалась В практике,В ветеринарной практике ситуация развивалась противоположенным образом .
9,распространяется с годов,В США с 1970-х годов все больше распространяется диагностика и лечение депрессии у домашних питомцев .


In [ ]:
95, из них 38 верно (исключая именительный). Кажется, это слишком мало, чтобы использовать смешанный подход

In [7]:
def ngrams_simple_case(conllu_sentences): ##Нграммы глаг. + (возможно) предлог + слово, имеющее падеж
    found_cases = pd.DataFrame({'sample': [], 'with_preposition': [], 'case': [], 'sentence': []})
    for sentence in conllu_sentences:
        sentence_forms = ' '.join([i['form'] for i in sentence])
        for i in range(len(sentence)-1):
            try:
                if sentence[i]['upostag'] == 'VERB':
                    if sentence[i+1]['upostag'] == 'ADP':
                        sample = ' '.join([sentence[j]['form'] for j in range(i, i+3)])
                        case = sentence[i+2]['feats']['Case']
                        found_cases.loc[len(found_cases)] = [sample, True, case, sentence_forms]
                    else:
                        sample = ' '.join([sentence[j]['form'] for j in range(i, i+2)])
                        case = sentence[i+1]['feats']['Case']
                        if case != 'Nom':
                            found_cases.loc[len(found_cases)] = [sample, False, case, sentence_forms]
            except:
                pass
    return found_cases

In [8]:
pd.set_option("display.max_rows",1000)

In [81]:
ngrams_found = ngrams_simple_case(pets_depression_sents)
ngrams_found

,sample,with_preposition,case,sentence
0,сделанные этологами,0.0,Ins,"В статье анализируются свидетельства о депрессивных расстройствах животных , сделанные этологами и психиатрами в XX ..."
1,Сложившаяся в 1980-е,1.0,Acc,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к..."
2,ставит во главу,1.0,Acc,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к..."
3,угла субъективные,0.0,Acc,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к..."
4,доминирующим подходом,0.0,Ins,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
5,имеющее физиологическую,0.0,Acc,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
6,происходить процессы,0.0,Acc,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."
7,сопутствуют депрессивному,0.0,Dat,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."
8,воспроизведены в лабораторных,1.0,Loc,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."
9,воспроизводящих депрессию,0.0,Acc,"Однако очевидная валидность экспериментов , воспроизводящих депрессию у животных , стала ставиться под сомнение ."


Найдено 282+179, из них правильно 129 + 129 правильных

Всего в тексте было 162 случая, которые можно найти при верной разметке, и 78 случаев, которые были бы найдены при идеальной синтаксической разметки (без учёта эллипсиса) - если считать только глаголы.

In [9]:
def get_next_ignore_part(sentence, word):
    try:
        next_word = sentence[word['id']]
        if next_word['upostag'] == 'PART':
            print(next_word['form'])
            next_word = get_next_ignore_part(sentence, next_word)
            print(' '.join([word['form'] for word in sentance]))
        return next_word
    except:
        return None

In [16]:
def ngrams_ignore_part(conllu_sentences):
    found_cases = pd.DataFrame({'sample': [], 'with_preposition': [], 'case': [], 'sentence': []})
    for sentence in conllu_sentences:
        sentence_forms = ' '.join([i['form'] for i in sentence])
        for i in range(len(sentence)-1):
            try:
                if sentence[i]['upostag'] == 'VERB':
                    next_word = get_next_ignore_part(sentence, sentence[i])
                    if next_word['upostag'] == 'ADP':
                        next_nextword = get_next_ignore_part(sentence, next_word)
                        sample = ' '.join([j['form'] for j in [sentence[i], next_word, next_nextword]])
                        case = next_nextword['feats']['Case']
                        found_cases.loc[len(found_cases)] = [sample, True, case, sentence_forms]
                    else:
                        case = next_word['feats']['Case']
                        if case != 'Nom':
                            found_cases.loc[len(found_cases)] = [sample, False, case, sentence_forms]
            except Exception as e:
                pass
                #print(e)
    return found_cases

In [33]:
ngrams_ignore_part_found

,sample,with_preposition,case,sentence
0,сделанные этологами,0.0,Ins,"В статье анализируются свидетельства о депрессивных расстройствах животных , сделанные этологами и психиатрами в XX ..."
1,Сложившаяся в 1980-е,1.0,Acc,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к..."
2,ставит во главу,1.0,Acc,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к..."
3,угла субъективные,0.0,Acc,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к..."
4,доминирующим подходом,0.0,Ins,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
5,имеющее физиологическую,0.0,Acc,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
6,происходить процессы,0.0,Acc,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."
7,сопутствуют депрессивному,0.0,Dat,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."
8,воспроизведены в лабораторных,1.0,Loc,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."
9,воспроизводящих депрессию,0.0,Acc,"Однако очевидная валидность экспериментов , воспроизводящих депрессию у животных , стала ставиться под сомнение ."


Посмотрим, какие добавились последовательности, когда мы стали пропускать частицы

In [34]:
baseline = set(ngrams_found['sample'])

In [35]:
n = 0
for sample in ngrams_ignore_part_found['sample']:
    if sample not in baseline:
        print(sample)
        n += 1  

вызваны физическими
страдать животное
становятся подопытными
осуществляться за счет
маркирует «поломку»
экстраполирует на депрессию
описывает социальный


Проверка разметки падежей: извлекли все существительные и падежные тэги, скачали таблицу. Если тэг неверный, вручную подписали правильный и загрузили обратно

In [4]:
cases = pd.DataFrame({'noun':[], 'case':[], 'context': []})
for sent in pets_depression_sents:
    nouns = sent.filter(upostag='NOUN')
    if nouns:
        sent_forms = ' '.join([word['form'] for word in sent])
        for noun in nouns:
            cases.loc[len(cases)] = [noun['form'], noun['feats']['Case'], sent_forms]
    

In [6]:
cases.to_csv('evaluate_cases.csv', encoding='Windows-1251')

In [60]:
cases = pd.read_csv('evaluate_cases_with_correct_tags.csv')

In [14]:
cases['correct_case'].count()/len(cases)

0.07955596669750231

In [19]:
cases['correct_case'].count()

86

In [59]:
len(cases)

1081

In [61]:
1-(86-33)/(1081-72)

0.9474727452923687

In [35]:
cases_mistakes = cases.dropna()
cases_mistakes

,Unnamed: 0,noun,case,correct_case,context
12,12,история,Nom,Acc,"В статье анализируются свидетельства о депрессивных расстройствах животных , сделанные этологами и психиатрами в XX ..."
20,20,ощущение,Nom,Acc,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к..."
27,27,расстройство,Nom,Acc,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ..."
31,31,процессы,Acc,Nom,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де..."
45,45,животное»,Gen,Nom,В контексте психических расстройств перевод «животное — человек — животное» работал неудовлетворительно : не хватало...
54,54,«симптомы»,Gen,Nom,"Наблюдаемые «симптомы» плохо соотносились с критериями расстройства , предполагающими обращение к субъектным пережив..."
79,79,«узнавать»,Gen,not_noun,"Таким образом , существуют определенные способы «узнавать» депрессию у животных , хотя ее критерии напрямую не приме..."
83,83,«узнавание»,Gen,Nom,Это «узнавание» проявлялось и в animal model of depression .
86,86,«узнавание»,Gen,Nom,"В статье делается предположение , что это «узнавание» имеет социальную природу : состояние депрессии приписывается ж..."
91,91,поломке,Dat,Loc,"В статье делается предположение , что это «узнавание» имеет социальную природу : состояние депрессии приписывается ж..."


In [46]:
cases_to_analyze = pd.DataFrame({})
for word in cases_mistakes['noun']:
    if len(re.findall('[«»]', word)) > 0:
        counter += 1

In [47]:
counter

33

In [48]:
counter = 0 ## В системе кавычки будут заменены при предобработке, а сейчас создают дополнительные ошибки
for word in cases['noun']:
    if len(re.findall('[«»]', word)) > 0:
        counter += 1
counter

72

In [67]:
cases_mistakes['should_be_analized'] = cases['noun'].apply(lambda x: len(re.findall('[«»]', x))==0)

C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [70]:
filtered_mistakes = cases_mistakes[cases_mistakes['should_be_analized']==True]

In [78]:
filtered_mistakes

,Unnamed: 0,noun,case,correct_case,context,should_be_analized
12,12,история,Nom,Acc,"В статье анализируются свидетельства о депрессивных расстройствах животных , сделанные этологами и психиатрами в XX ...",True
20,20,ощущение,Nom,Acc,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к...",True
27,27,расстройство,Nom,Acc,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ...",True
31,31,процессы,Acc,Nom,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де...",True
91,91,поломке,Dat,Loc,"В статье делается предположение , что это «узнавание» имеет социальную природу : состояние депрессии приписывается ж...",True
113,113,практике,Dat,Loc,"Он обращается к противоречию , возникающему в современной ему ветеринарной практике .",True
117,117,симптомы,Nom,Acc,"Ветеринары наблюдали в поведении животных симптомы и болезни , для описания которых заимствовали психиатрические тер...",True
118,118,болезни,Nom,Acc,"Ветеринары наблюдали в поведении животных симптомы и болезни , для описания которых заимствовали психиатрические тер...",True
120,120,термины,Nom,Acc,"Ветеринары наблюдали в поведении животных симптомы и болезни , для описания которых заимствовали психиатрические тер...",True
123,123,собак,Acc,Gen,"Ветеринары наблюдали в поведении животных симптомы и болезни , для описания которых заимствовали психиатрические тер...",True


In [75]:
from collections import Counter
mistakes_types = Counter([row['case'] + ' ' + row['correct_case'] for index, row in filtered_mistakes.iterrows()])

In [77]:
len(filtered_mistakes)

52

In [76]:
mistakes_types 

Counter({'Nom Acc': 17,
         'Acc Nom': 7,
         'Dat Loc': 2,
         'Acc Gen': 4,
         'Gen Nom': 3,
         'Nom Gen': 3,
         'Gen Acc': 5,
         'Nom not_noun': 2,
         'Loc Acc': 1,
         'Acc not_noun': 1,
         'Gen Dat': 2,
         'Dat Ins': 1,
         'Par Dat': 1,
         'Gen Loc': 1,
         'Nom Loc': 1,
         'Acc Dat': 1})

Оценка согласования

In [83]:
def get_number_or_gender(word):
    try:
        if word['feats']['Number'] == 'Plur':
            agreement = 'Plur'
        else:
            agreement = word['feats']['Gender']
    except:
        agreement = 'no_NoG'
    return agreement

In [84]:
def get_case(word):
    try:
        return word['feats']['Case']
    except:
        return 'noCase'

In [85]:
def get_agreement_features(sents):
    found_adjectives = pd.DataFrame({'sentence': [], 'adj': [], 'adj_numorgender': [], 'adj_case':[], 'head':[], 'head_numorgender': [], 'head_case': []})
    for sent in sents:
        sent_form = ' '.join([i['form'] for i in sent])
        adjectives = sent.filter(upostag='ADJ')
        if adjectives:
            for adjective in adjectives:
                head_data = sent[adjective['head']-1]
                if head_data['upostag'] == 'NOUN':
                    adj_form = adjective['form']
                    adj_n_or_gender = get_number_or_gender(adjective)
                    adj_case = get_case(adjective)
                    head_form = head_data['form']
                    head_n_or_gender = get_number_or_gender(head_data)
                    head_case = get_case(head_data)
                    adj_feats = [adj_n_or_gender, adj_case]
                    head_feats = [head_n_or_gender, head_case]
                    found_adjectives.loc[len(found_adjectives)] = [sent_form, adj_form, adj_n_or_gender, adj_case, head_form, head_n_or_gender, head_case]
    return found_adjectives

In [86]:
agreement_features = get_agreement_features(pets_depression_sents)

In [87]:
agreement_features

,sentence,adj,adj_numorgender,adj_case,head,head_numorgender,head_case
0,Животная тоска : концептуализация депрессии в психиатрии и ветеринарии doi : 10.22394 / 2074 - 0492 - 2019 - 3 -57-7...,Животная,Fem,Nom,тоска,Fem,Nom
1,"В статье анализируются свидетельства о депрессивных расстройствах животных , сделанные этологами и психиатрами в XX ...",депрессивных,Plur,Loc,расстройствах,Plur,Loc
2,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к...",1980-е,Plur,Acc,концептуализация,Fem,Nom
3,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к...",психиатрическая,Fem,Nom,концептуализация,Fem,Nom
4,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к...",субъективные,Plur,Acc,критерии,Plur,Acc
5,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ...",биологическая,Fem,Nom,модель,Fem,Nom
6,"Одновременно с этим доминирующим подходом к объяснению депрессии становится биологическая модель , в рамках которой ...",физиологическую,Fem,Acc,природу,Fem,Acc
7,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де...",схожие,Plur,Acc,процессы,Plur,Acc
8,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де...",депрессивному,Neut,Dat,расстройству,Neut,Dat
9,"Это означало , что в организме животных могут происходить процессы , схожие с теми , что вызывают или сопутствуют де...",лабораторных,Plur,Loc,экспериментах,Plur,Loc


Посмотрим на ложные ошибки

In [90]:
wrong_numorgender = agreement_features[agreement_features['adj_numorgender'] != agreement_features['head_numorgender']]
wrong_numorgender

,sentence,adj,adj_numorgender,adj_case,head,head_numorgender,head_case
2,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к...",1980-е,Plur,Acc,концептуализация,Fem,Nom
12,В контексте психических расстройств перевод «животное — человек — животное» работал неудовлетворительно : не хватало...,«животное,Neut,Nom,перевод,Masc,Nom
24,Для уточнения этого тезиса рассматривается интеракционистская и этнометодологическая подходы к описанию взаимодейств...,интеракционистская,Fem,Nom,подходы,Plur,Nom
47,"Тогда , продолжает Линдсей , болезни , что наблюдал он и его коллеги , должны быть квалифицированы как психические р...",«суицидальные,Plur,Nom,меланхолии»,Fem,Gen
54,"Таким образом , признание , что животное может страдать психическим расстройством , автоматически ставило для Линдсе...",должны,Plur,noCase,вопрос,Masc,Nom
77,"Его несколько раз переводили из клетки в клетку к разным самкам , после чего исследователь заметил за ним странности...",подобное,Neut,Acc,депрессии,Fem,Gen
79,В обоих случаях исследователи фиксируют два принципиальных момента : животное не идет на контакт ( причем именно с л...,принципиальных,Plur,Gen,момента,Masc,Gen
91,Депрессивные кошки действительно похожи на маленькие безжизненные комки» [ Braitman 2013 : 177 ] .,маленькие,Plur,Acc,комки»,Fem,Gen
92,Депрессивные кошки действительно похожи на маленькие безжизненные комки» [ Braitman 2013 : 177 ] .,безжизненные,Plur,Acc,комки»,Fem,Gen
132,"Позже он создал модель депрессии , согласно которой психическое расстройство возникало вследствие изоляции детенышей...",«социального,Masc,Gen,контакта»,Fem,Gen


1, 2 - переход прилагательного в существительное
3 - действительно ошибка
4- неверный тэг существительного
5, 6 - неверная синтаксическая связь 
7 - cчетная конструкция
8 - 11 неверный тэг существительного
12 - неверная разметка 
13 - не прилагательное
14 - неверная связь
15 - счетная конструкция
16 - неверная разметка
17  - неверная разметка
18 - неверная связь

In [92]:
wrong_case = agreement_features[agreement_features['adj_case'] != agreement_features['head_case']]
wrong_case

,sentence,adj,adj_numorgender,adj_case,head,head_numorgender,head_case
2,"Сложившаяся в 1980-е психиатрическая концептуализация депрессии ставит во главу угла субъективные критерии , такие к...",1980-е,Plur,Acc,концептуализация,Fem,Nom
28,"Он обращается к противоречию , возникающему в современной ему ветеринарной практике .",современной,Fem,Loc,практике,Fem,Dat
47,"Тогда , продолжает Линдсей , болезни , что наблюдал он и его коллеги , должны быть квалифицированы как психические р...",«суицидальные,Plur,Nom,меланхолии»,Fem,Gen
51,"Это означало , что отношения с животными и их лечение должны строиться на «законах доброты» , то есть на тех же прин...",должны,Plur,noCase,принципах,Plur,Loc
54,"Таким образом , признание , что животное может страдать психическим расстройством , автоматически ставило для Линдсе...",должны,Plur,noCase,вопрос,Masc,Nom
77,"Его несколько раз переводили из клетки в клетку к разным самкам , после чего исследователь заметил за ним странности...",подобное,Neut,Acc,депрессии,Fem,Gen
91,Депрессивные кошки действительно похожи на маленькие безжизненные комки» [ Braitman 2013 : 177 ] .,маленькие,Plur,Acc,комки»,Fem,Gen
92,Депрессивные кошки действительно похожи на маленькие безжизненные комки» [ Braitman 2013 : 177 ] .,безжизненные,Plur,Acc,комки»,Fem,Gen
160,"Таким образом , если камера для изоляции выступала в качестве материального воплощения человеческих переживаний , то...",другими,Plur,Ins,взаимодействие,Neut,Acc
177,Участники эксперимента и само оборудование наделялись эмоциональными характеристикам .,эмоциональными,Plur,Ins,характеристикам,Plur,Dat


1 - связь
2 - форма сущ (омо)
3 - форма сущ
4, 5 - краткое + связь
6 - связь
7, 8 - форма
9 - связь
10 - ошибка
11 - краткое
12 - краткое + форма
13 - форма
14 - форма
15-17 - форма
 18 - связь

In [78]:
len(agreement_features[agreement_features['adj_case'] != agreement_features['head_case']])

41

In [97]:
def get_agreement_features_not_noun(sents):
    found_adjectives = pd.DataFrame({'sentence': [], 'adj': [], 'adj_numorgender': [], 'adj_case':[], 'head':[]})
    for sent in sents:
        sent_form = ' '.join([i['form'] for i in sent])
        adjectives = sent.filter(upostag='ADJ')
        if adjectives:
            for adjective in adjectives:
                head_data = sent[adjective['head']-1]
                if head_data['upostag'] != 'NOUN':
                    adj_form = adjective['form']
                    adj_n_or_gender = get_number_or_gender(adjective)
                    adj_case = get_case(adjective)
                    head_form = head_data['form']
                    found_adjectives.loc[len(found_adjectives)] = [sent_form, adj_form, adj_n_or_gender, adj_case, head_form]
    return found_adjectives

In [98]:
not_noun_dependences = get_agreement_features_not_noun(pets_depression_sents)

In [99]:
not_noun_dependences

,sentence,adj,adj_numorgender,adj_case,head
0,"Ветеринары и владельцы домашних животных оказываются способны приписывать депрессивное расстройство животным , опира...",способны,Plur,noCase,оказываются
1,"Таким образом , существуют определенные способы «узнавать» депрессию у животных , хотя ее критерии напрямую не приме...",применимы,Plur,noCase,существуют
2,Для уточнения этого тезиса рассматривается интеракционистская и этнометодологическая подходы к описанию взаимодейств...,этнометодологическая,Fem,Nom,интеракционистская
3,Тем не менее они отказывались признавать возможность безумия у животных : «Хотя все ветеринары признают определенные...,безумными,Plur,Ins,казаться
4,"Он пишет , что , если животные могут сходить с ума так же , как и люди , то и причины их болезней также должны быть ...",должны,Plur,noCase,пишет
5,"Он пишет , что , если животные могут сходить с ума так же , как и люди , то и причины их болезней также должны быть ...",сходны,Plur,noCase,должны
6,"Он пишет , что , если животные могут сходить с ума так же , как и люди , то и причины их болезней также должны быть ...",человеческими,Plur,Ins,сходны
7,"Значит , психические расстройства у животных могут быть вызваны не только физическими , но и ментальными причинами :...",ментальными,Plur,Ins,физическими
8,"Значит , психические расстройства у животных могут быть вызваны не только физическими , но и ментальными причинами :...",телесном,Masc,Loc,мы
9,"Значит , психические расстройства у животных могут быть вызваны не только физическими , но и ментальными причинами :...",подвержены,Plur,noCase,думают


In [ ]:
0 - от глагола
1 - неверная связь 
2 - однородные члены (прил.)
3 - от глагола
4 - краткая форма, от др. части предложения
5 - от предиката (кр.)
6 - эллипсис
7 - однородные 
8 - неверная связь
9 - от другой основы (глагол)
10 - те самые
11 - от другой основы
12 - управление
13, 14 - однородные 
15 - другая основа
16 - другая основа
17 - согласование
18 - от глагола (выглядят противоречивыми)
19 - неверно
20 - другая основа
21 - 23 - зависит от пунктуации, простое предложение, сказуемое
24 - другая основа
25 - - зависит от пунктуации, простое предложение, сказуемое
26 - управление
27 - зависит от пунктуации, простое предложение, сказуемое
28 - неверно
29 - управление
30 - ошибка
31 - подлежащее
32 - ошибка
33 - сравнительная степень
34 - ошибка
35 - однородные
36 - от сказуемого
37 - управление
38 - ошибка
39 - 'невозможно'
40 - от глагола
41 - другая основа
42 - управление
43 - другая основа
44 - управление
45 - управление
46 - другая основа
47 - подлежащее
48 - управление
49 - управление
50 - подлежащее + ошибка
51 - ошибка 

Проверка дополнительных условий

Выявление предложений с отрицанием

In [26]:
for sent in pets_depression_sents:
    forms = [word['form'] for word in sent]
    if 'не' in forms:
        print(' '.join(forms))

В контексте психических расстройств перевод «животное — человек — животное» работал неудовлетворительно : не хватало оснований для проведения аналогии между состоянием животного в эксперименте и человеком в депрессии .
Таким образом , существуют определенные способы «узнавать» депрессию у животных , хотя ее критерии напрямую не применимы к ним .
Тем не менее они отказывались признавать возможность безумия у животных : «Хотя все ветеринары признают определенные ментальные феномены , сопутствующие некоторым заболеваниям у «низших животных» , тем не менее они отрицают их параллелизм с безумием человека на том основании , что животные не обладают здоровым сознанием , а , значит , не могут обладать больным» [ Lindsay 1871 : 182 ] ( здесь и далее перевод автора ) Действия животных могут быть или казаться безумными в силу какого-либо заболевания , но животные не могут сойти с ума : у них нет сознания , которое могло бы помутиться .
Значит , психические расстройства у животных могут быть вызва

Если искать просто в предложении, выявляется много лишних случаев. Будем искать 'не' именно при глаголе

In [27]:
for sent in pets_depression_sents:
    tokens_ne = sent.filter(form='не')
    for part in tokens_ne:
        if sent[part['head']-1]['upostag'] == 'VERB':
            print(' '.join([word['form'] for word in sent]))

В контексте психических расстройств перевод «животное — человек — животное» работал неудовлетворительно : не хватало оснований для проведения аналогии между состоянием животного в эксперименте и человеком в депрессии .
Тем не менее они отказывались признавать возможность безумия у животных : «Хотя все ветеринары признают определенные ментальные феномены , сопутствующие некоторым заболеваниям у «низших животных» , тем не менее они отрицают их параллелизм с безумием человека на том основании , что животные не обладают здоровым сознанием , а , значит , не могут обладать больным» [ Lindsay 1871 : 182 ] ( здесь и далее перевод автора ) Действия животных могут быть или казаться безумными в силу какого-либо заболевания , но животные не могут сойти с ума : у них нет сознания , которое могло бы помутиться .
Тем не менее они отказывались признавать возможность безумия у животных : «Хотя все ветеринары признают определенные ментальные феномены , сопутствующие некоторым заболеваниям у «низших живо